# ChatBot Finanças Expert AI

# 1. Instalação das Bibliotecas:

In [ ]:
!pip install -q -U google-generativeai
!pip install pdfplumber
!pip install nltk
!pip install numpy

# Chatbot Assistente de Finanças com Google Gemini Pro a partir de um PDF de Guia de Finanças

O Código lê um PDF enviado para a raiz do sistema, ele tokeniza os textos e inseri no prompt do contexto da conversa do Gemini com  usuário. Interessante é que ele resumi textos complexos como esse Guia e traduz para o usuário poder conversar com o Chatbot. O grande desafio é a qualidade desse resumo e o maximo numero de Tokens que voce pode inserir no prompt, fizemos esse controle em max_user_input_length=256, pode-se testar diferente valores para testar o modelo.

Sugestões e Melhorias, faça o Fork desse projeto e pode enviar melhorias para mim em meus contatos!

Claudio M. Antonio - Dev

In [65]:
import google.generativeai as genai
import pdfplumber
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from scipy.spatial.distance import cosine
import numpy as np
from google.colab import userdata
# Carregar a chave da API Gemini (necessário para GenAI)
api_key = userdata.get('GEMINI_KEY')
genai.configure(api_key=api_key)  # Mantenha se quiser usar o GenAI

def preprocess_pdf(pdf_path):
    """
    Função para pré-processar o PDF.

    Args:
        pdf_path: Caminho para o arquivo PDF.

    Returns:
        str: Texto extraído do PDF.
    """
    with open(pdf_path, 'rb') as pdf_file:
        pdf = pdfplumber.open(pdf_file)
        pdf_text = ''
        for page in pdf.pages:
            pdf_text += page.extract_text()
    return pdf_text

def summarize_text(text, ratio=0.2):
    """
    Função para resumir o texto usando a biblioteca NLTK.

    Args:
        text: Texto a ser resumido.
        ratio: Razão de compressão do resumo (entre 0 e 1).

    Returns:
        str: Texto resumido.
    """
    # Download necessário apenas na primeira execução
    nltk.download('stopwords')
    nltk.download('punkt')

    stop_words = stopwords.words('portuguese')
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    sentences = sent_tokenize(text)

    sentence_vectors = []
    for sentence in sentences:
        sentence_words = word_tokenize(sentence.lower())
        sentence_vec = []
        for word in filtered_text:
            if word in sentence_words:
                sentence_vec.append(1)
            else:
                sentence_vec.append(0)
        sentence_vectors.append(sentence_vec)

    sentence_scores = np.apply_along_axis(lambda vec: cosine(vec, np.sum(sentence_vectors, axis=0)), axis=1, arr=np.array(sentence_vectors))
    top_sentences = np.argsort(sentence_scores)[:int(len(sentences) * ratio)]
    summary = ' '.join([sentences[idx] for idx in top_sentences])
    return summary

def chatbot_with_gemini(pdf_path, max_user_input_length=256):
    """
    Função principal para interagir com o chatbot Gemini.

    Args:
        pdf_path: Caminho para o arquivo PDF.
    """
    # Pré-processar o PDF
    preprocessed_text = preprocess_pdf(pdf_path)

    # Resumir o texto do PDF
    summary_text = summarize_text(preprocessed_text)

    # Iniciar a sessão de chat com o modelo Gemini
    chat = genai.GenerativeModel('gemini-pro').start_chat(history=[])

    # Enviar o prompt com o resumo do PDF como o texto inicial
    response = chat.send_message(summary_text)



    # Exibir o prompt inicial do chatbot
    print(f"Chatbot: {response.text}")
    # Perguntar ao usuário o que deseja saber sobre Finanças
    print("Chatbot: O que você deseja saber sobre Finanças? Vamos utilizar o PDF guia de finanças como guia.")


    # Loop principal da conversa
    # Loop principal da conversa
    while True:
      # Obter a entrada do usuário
      user_input = input("Usuário: ")

      # Truncate user input if it exceeds the limit
      if len(user_input) > max_user_input_length:
        user_input = user_input[:max_user_input_length]

      # Processar a entrada do usuário e gerar uma resposta
      response = chat.send_message(user_input)

      # Exibir a resposta do chatbot
      print(f"Chatbot: {response.text}")

import requests

# URL do arquivo PDF no GitHub
pdf_url = "https://github.com/upcarreira/Meu-Portfolio/raw/main/guia-financas.pdf"

# Caminho onde o PDF será salvo localmente
pdf_local_path = "guia-financas.pdf"

# Baixar o PDF
response = requests.get(pdf_url)
with open(pdf_local_path, "wb") as pdf_file:
    pdf_file.write(response.content)

# Chamar a função chatbot_with_gemini() com o caminho local do PDF
chatbot_with_gemini(pdf_local_path)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Chatbot: **Insights de Warren Buffett, Ray Dalio e Aswath Damodaran**

**Warren Buffett**

* Enfatiza investir em empresas sólidas com vantagens competitivas duradouras.
* Manter uma abordagem de longo prazo para investimentos.
* Berkshire Hathaway: conglomerado diversificado com negócios em diversos setores.
* Círculo de competência: investe em áreas que entende completamente.
* American Express: outra empresa de longa data no portfólio de Buffett, se beneficia de sua forte marca e rede global de pagamentos.

**Ray Dalio**

* Destaca a importância da diversificação de portfólio.
* Compreender os ciclos econômicos é crucial para o sucesso financeiro.
* Bridgewater Associates: maior fundo de hedge do mundo, conhecido por sua abordagem sistemática de investimentos.

**Aswath Damodaran**

* Especialista em finanças que oferece perspectivas únicas sobre alocação de ativos.
* Enfatiza a importância de uma abordagem estratégica e diversificada.
* Ferramentas de avaliação: modelos de fluxo de

KeyboardInterrupt: Interrupted by user